In [1]:
import pandas as pd
import numpy as np
import time
import sys

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression  
from sklearn import svm

module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Cohort_selection/'
if module_path not in sys.path:
    sys.path.append(module_path)

from cohort_loader import *
import eicu_year_process

module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/evaluation/'
if module_path not in sys.path:
    sys.path.append(module_path)

import get_hospital_eicu

import split
from imp import reload
reload(split)

import Multiclass_evaluation
reload(Multiclass_evaluation)

c:\Users\DAHS\anaconda3\envs\DL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'Multiclass_evaluation' from 'c:\\Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/evaluation\\Multiclass_evaluation.py'>

In [2]:
mimic_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/mimic_analysis.csv.gz'
eicu_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/eicu_analysis.csv.gz'

mimic = pd.read_csv(mimic_path, compression = 'gzip')
mimic = mimic.drop('Shock_next_12h', axis = 1)

eicu = pd.read_csv(eicu_path, compression = 'gzip')
eicu = eicu.drop('Shock_next_12h', axis = 1)

dataset_mimic = mimic[~(mimic['gender']==2)].reset_index(drop=True)

dataset_mimic.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset_mimic.fillna(0, inplace=True) 
dataset_mimic = dataset_mimic[~(dataset_mimic['Case']=='event')]

eventset = mimic[(mimic['Case']=='event')].reset_index(drop=True)
dataset_mimic['Case'] = pd.to_numeric(dataset_mimic['Case'], errors='coerce')

mimic_train, mimic_valid = data_split(dataset_mimic, 9040, 0.9, Threshold=0.05, n_trial=1, mode = 'mimic')

eicu_t = eicu_year_process.matching_patient(eicu)
dataset_eicu = eicu_t[~(eicu_t['gender']==2)].reset_index(drop=True)
dataset_eicu.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset_eicu.fillna(0, inplace=True) 

eicu_test = dataset_eicu[~(dataset_eicu['Case']=='event')]
event_eicu = dataset_eicu[(dataset_eicu['Case']=='event')]
eicu_test['Case'] = pd.to_numeric(eicu_test['Case'], errors='coerce')
hosp_id, eicu_test = get_hospital_eicu.hospital(eicu_test)

eicu_type = get_hospital_eicu.eicu_subgroup(eicu_test)
unitadmitsource, unittype, unitstaytype = get_hospital_eicu.make_eicu_dataset(eicu_type)

========== 데이터셋 분할 정보 ==========
데이터셋 비율: 학습 = 0.90, 테스트 = 0.10
학습셋 클래스 비율: Case
1.0    473941
2.0      5128
3.0     11145
4.0     19613
Name: count, dtype: int64
테스트셋 클래스 비율: Case
1.0    53103
2.0      512
3.0      873
4.0     1590
Name: count, dtype: int64
--------------------------------------
========== 클래스 비율 ==========
학습셋 클래스 비율: 0.93:0.01:0.02:0.04
테스트셋 클래스 비율: 0.95:0.01:0.02:0.03
--------------------------------------
========== 환자 및 체류 정보 ==========
학습셋 환자 수: 2907
테스트셋 환자 수: 329
학습셋 체류 수: 2961
테스트셋 체류 수: 330
--------------------------------------
========== 실험 설정 ==========
분할 시드: 9040
학습 비율: 0.9
임계값: 0.05
--------------------------------------
========== 실행 결과 ==========
총 소요 시간(초): 0.38
시도한 시행 횟수: 0


In [242]:
module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/sub task prediction/'

if module_path not in sys.path:
    sys.path.append(module_path)

import incorperate_subtask_label
reload(incorperate_subtask_label)

<module 'incorperate_subtask_label' from 'c:\\Users\\DAHS\\Desktop\\ECP_CONT\\ECP_SCL\\Experiment(Supervised Learning)\\sub task prediction\\incorperate_subtask_label.py'>

In [181]:
mort_mimic_train, mort_mimic_valid, mort_eicu_test = incorperate_subtask_label.mortality_prediction_DATA(mimic_train, mimic_valid, eicu_test)

In [43]:
los_mimic_train, los_mimic_valid, los_eicu_test = incorperate_subtask_label.LOS_prediction_DATA(mimic_train, mimic_valid, eicu_test)

In [204]:
ards4h_mimic_train, ards4h_mimic_valid, ards4h_eicu_test = incorperate_subtask_label.ARDS4_prediction_DATA(mimic_train, mimic_valid, eicu_test)

In [225]:
ARDS4h_event_train = ards4h_mimic_train[ards4h_mimic_train['Annotation_ARDS']=='ARDS']
ARDS4h_event_train['ARDS_next_4h']='event'
ARDS4h_event_valid = ards4h_mimic_valid[ards4h_mimic_valid['Annotation_ARDS']=='ARDS']
ARDS4h_event_valid['ARDS_next_4h']='event'
ARDS4h_event_test = ards4h_eicu_test[ards4h_eicu_test['Annotation_ARDS']=='ARDS']
ARDS4h_event_test['ARDS_next_4h']='event'

In [207]:
ards8h_mimic_train, ards8h_mimic_valid, ards8h_eicu_test = incorperate_subtask_label.ARDS8_prediction_DATA(mimic_train, mimic_valid, eicu_test)

In [235]:
ARDS8h_event_train = ards8h_mimic_train[ards8h_mimic_train['Annotation_ARDS']=='ARDS']
ARDS8h_event_train['ARDS_next_8h']='event'
ARDS8h_event_valid = ards8h_mimic_valid[ards8h_mimic_valid['Annotation_ARDS']=='ARDS']
ARDS8h_event_valid['ARDS_next_8h']='event'
ARDS8h_event_test = ards8h_eicu_test[ards8h_eicu_test['Annotation_ARDS']=='ARDS']
ARDS8h_event_test['ARDS_next_8h']='event'

In [243]:
sic4h_mimic_train, sic4h_mimic_valid, sic4h_eicu_test = incorperate_subtask_label.SIC4_prediction_DATA(mimic_train, mimic_valid, eicu_test)

In [245]:
SIC4h_event_train = sic4h_mimic_train[sic4h_mimic_train['Annotation_SIC']=='SIC']
SIC4h_event_train['SIC_next_4h']='event'
SIC4h_event_valid = sic4h_mimic_valid[sic4h_mimic_valid['Annotation_SIC']=='SIC']
SIC4h_event_valid['SIC_next_4h']='event'
SIC4h_event_test = sic4h_eicu_test[sic4h_eicu_test['Annotation_SIC']=='SIC']
SIC4h_event_test['SIC_next_4h']='event'

In [246]:
sic8h_mimic_train, sic8h_mimic_valid, sic8h_eicu_test = incorperate_subtask_label.SIC8_prediction_DATA(mimic_train, mimic_valid, eicu_test)

In [247]:
SIC8h_event_train = sic8h_mimic_train[sic8h_mimic_train['Annotation_SIC']=='SIC']
SIC8h_event_train['SIC_next_8h']='event'
SIC8h_event_valid = sic8h_mimic_valid[sic8h_mimic_valid['Annotation_SIC']=='SIC']
SIC8h_event_valid['SIC_next_8h']='event'
SIC8h_event_test = sic8h_eicu_test[sic8h_eicu_test['Annotation_SIC']=='SIC']
SIC8h_event_test['SIC_next_8h']='event'

In [212]:
reload(split)

<module 'split' from 'c:\\Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/evaluation\\split.py'>

In [199]:
ards4h_mimic_train[ards4h_mimic_train['Annotation_ARDS']=='no_ARDS'].ARDS_next_4h.value_counts(normalize=True)

ARDS_next_4h
0.0    0.931524
1.0    0.068476
Name: proportion, dtype: float64

In [250]:
X_train_mort, y_train_mort, _ = split.split_X_Y_MORT(mort_mimic_train[~(mort_mimic_train['death']=='event')], mode = 'mimic')
X_valid_mort, y_valid_mort, valid_output_mort = split.split_X_Y_MORT(mort_mimic_valid[~(mort_mimic_valid['death']=='event')], mode = 'mimic')

X_train_los, y_train_los, _ = split.split_X_Y_LOS(los_mimic_train, mode = 'mimic')
X_valid_los, y_valid_los, valid_output_los = split.split_X_Y_LOS(los_mimic_valid, mode = 'mimic')

X_train_ards4h, y_train_ards4h, _ = split.split_X_Y_ARDS4h(ards4h_mimic_train[ards4h_mimic_train['Annotation_ARDS']=='no_ARDS'], mode = 'mimic')
X_valid_ards4h, y_valid_ards4h, valid_output_ards4h = split.split_X_Y_ARDS4h(ards4h_mimic_valid[ards4h_mimic_valid['Annotation_ARDS']=='no_ARDS'], mode = 'mimic')

X_train_ards8h, y_train_ards8h, _ = split.split_X_Y_ARDS8h(ards8h_mimic_train[ards8h_mimic_train['Annotation_ARDS']=='no_ARDS'], mode = 'mimic')
X_valid_ards8h, y_valid_ards8h, valid_output_ards8h = split.split_X_Y_ARDS8h(ards8h_mimic_valid[ards8h_mimic_valid['Annotation_ARDS']=='no_ARDS'], mode = 'mimic')

X_train_sic4h, y_train_sic4h, _ = split.split_X_Y_SIC4h(sic4h_mimic_train[sic4h_mimic_train['Annotation_SIC']=='no_SIC'], mode = 'mimic')
X_valid_sic4h, y_valid_sic4h, valid_output_sic4h = split.split_X_Y_SIC4h(sic4h_mimic_valid[sic4h_mimic_valid['Annotation_SIC']=='no_SIC'], mode = 'mimic')

X_train_sic8h, y_train_sic8h, _ = split.split_X_Y_SIC8h(sic8h_mimic_train[sic8h_mimic_train['Annotation_SIC']=='no_SIC'], mode = 'mimic')
X_valid_sic8h, y_valid_sic8h, valid_output_sic8h = split.split_X_Y_SIC8h(sic8h_mimic_valid[sic8h_mimic_valid['Annotation_SIC']=='no_SIC'], mode = 'mimic')

In [253]:
reload(Multiclass_evaluation) 

<module 'Multiclass_evaluation' from 'c:\\Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/evaluation\\Multiclass_evaluation.py'>

In [239]:
mort_models, result_mort = Multiclass_evaluation.create_subtask(X_train_mort, y_train_mort, X_valid_mort, valid_output_mort, _,mode = 'mimic', type = 'binary', event_task = False)

|MIMIC-Validation|====================================================


,Model,Accuracy,AUPRC,F1 Score,Recall,Precision
0,lgbm,0.881533,0.410500,0.146840,0.082723,0.652893
1,rf,0.883985,0.310726,0.214847,0.128796,0.647368
2,dt,0.775455,0.166703,0.292107,0.375916,0.238856
3,lr,0.876758,0.197429,0.000000,0.000000,0.000000


----------------------------------------------------------------------


In [196]:
los_models, result_los = Multiclass_evaluation.create_subtask(X_train_los, y_train_los, X_valid_los, valid_output_los, _, mode = 'mimic', type = 'multi', event_task = False)

|MIMIC-Validation|====================================================


,Model,Accuracy,Macro AUPRC,F1 Score,Macro averaged precision,Macro averaged Recall
0,lgbm,0.5259,0.583333,0.1940,0.2873,0.2299
1,rf,0.4796,0.483088,0.2404,0.2483,0.2611
2,dt,0.3531,0.511390,0.2312,0.2316,0.2337
3,lr,0.4809,0.583451,0.1098,0.4135,0.1675


----------------------------------------------------------------------


In [232]:
ards4h_models, result_ards4h = Multiclass_evaluation.create_subtask(X_train_ards4h, y_train_ards4h, X_valid_ards4h, valid_output_ards4h, ARDS4h_event_valid, mode = 'mimic', type = 'binary', event_task='ARDS 4h')

|MIMIC-Validation|====================================================


,Model,Accuracy,AUPRC,Recall,Precision
0,lgbm,0.889317,0.504842,0.2105,0.916667
1,rf,0.883888,0.401885,0.4354,0.579805
2,dt,0.810333,0.193515,0.5742,0.300323
3,lr,0.874431,0.181386,0.0287,0.424242


----------------------------------------------------------------------


In [236]:
ards8h_models, result_ards8h = Multiclass_evaluation.create_subtask(X_train_ards8h, y_train_ards8h, X_valid_ards8h, valid_output_ards8h, ARDS8h_event_valid, mode = 'mimic', type = 'binary', event_task='ARDS 8h')

|MIMIC-Validation|====================================================


,Model,Accuracy,AUPRC,Recall,Precision
0,lgbm,0.822242,0.524101,0.3636,0.873418
1,rf,0.798949,0.460599,0.6746,0.529024
2,dt,0.620490,0.228960,0.7847,0.255884
3,lr,0.789142,0.270022,0.0478,0.411765


----------------------------------------------------------------------


In [254]:
sic4h_models, result_sic4h = Multiclass_evaluation.create_subtask(X_train_sic4h, y_train_sic4h, X_valid_sic4h, valid_output_sic4h, SIC4h_event_valid, mode = 'mimic', type = 'binary', event_task='SIC 4h')

|MIMIC-Validation|====================================================


,Model,Accuracy,AUPRC,Recall,Precision
0,lgbm,0.927870,0.322420,0.0247,0.750000
1,rf,0.923660,0.171342,0.1235,0.392157
2,dt,0.824586,0.090225,0.4074,0.137450
3,lr,0.927589,0.317256,0.0247,0.800000


----------------------------------------------------------------------


In [255]:
sic8h_models, result_sic8h = Multiclass_evaluation.create_subtask(X_train_sic8h, y_train_sic8h, X_valid_sic8h, valid_output_sic8h, SIC8h_event_valid, mode = 'mimic', type = 'binary', event_task='SIC 8h')

|MIMIC-Validation|====================================================


,Model,Accuracy,AUPRC,Recall,Precision
0,lgbm,0.874544,0.443275,0.0247,0.400000
1,rf,0.858546,0.232822,0.3827,0.340659
2,dt,0.791468,0.171564,0.5432,0.256956
3,lr,0.877631,0.420684,0.0370,1.000000


----------------------------------------------------------------------
